Welcome to part 5 of the Deep learning with Python, TensorFlow and Keras tutorial series. In the previous tutorial, we introduced TensorBoard, which is an application that we can use to visualize our model's training stats over time. In this tutorial, we're going to continue on that to exemplify how you might build a workflow to optimize your model's architecture.

To begin, let's think of a few things we could do to this model that we'd like to know.

The most basic things for us to modify are layers and nodes per layer, as well as 0, 1, or 2 dense layers. Let's test those things. How might we do this?

In [1]:
# A simple for-loop will do! For example:

import time

dense_layers = [0,1,2]
layer_sizes = [32, 64, 128]
conv_layers = [1, 2, 3]

for dense_layer in dense_layers:
    for layer_size in layer_sizes:
        for conv_layer in conv_layers:
            NAME = "{}-conv-{}-nodes-{}-dense-{}".format(conv_layer, layer_size, dense_layer, int(time.time()))
            print(NAME)


1-conv-32-nodes-0-dense-1542608880
2-conv-32-nodes-0-dense-1542608880
3-conv-32-nodes-0-dense-1542608880
1-conv-64-nodes-0-dense-1542608880
2-conv-64-nodes-0-dense-1542608880
3-conv-64-nodes-0-dense-1542608880
1-conv-128-nodes-0-dense-1542608880
2-conv-128-nodes-0-dense-1542608880
3-conv-128-nodes-0-dense-1542608880
1-conv-32-nodes-1-dense-1542608880
2-conv-32-nodes-1-dense-1542608880
3-conv-32-nodes-1-dense-1542608880
1-conv-64-nodes-1-dense-1542608880
2-conv-64-nodes-1-dense-1542608880
3-conv-64-nodes-1-dense-1542608880
1-conv-128-nodes-1-dense-1542608880
2-conv-128-nodes-1-dense-1542608880
3-conv-128-nodes-1-dense-1542608880
1-conv-32-nodes-2-dense-1542608880
2-conv-32-nodes-2-dense-1542608880
3-conv-32-nodes-2-dense-1542608880
1-conv-64-nodes-2-dense-1542608880
2-conv-64-nodes-2-dense-1542608880
3-conv-64-nodes-2-dense-1542608880
1-conv-128-nodes-2-dense-1542608880
2-conv-128-nodes-2-dense-1542608880
3-conv-128-nodes-2-dense-1542608880


So that's a lot of combinations. I will be running them all, you don't have to. If you have a decent GPU, you can install and use Tensorflow-GPU instead. If you want to learn how to do that, I have two tutorials doing it:

TensorFlow-GPU on Ubuntu
TensorFlow-GPU on Windows
Both videos are for an older version of TF, but the methodology for getting Tensorflow-GPU is fairly straight forward. You do a pip install tensorflow-gpu, then download the Cuda Toolkit, and then CuDNN. Install Cuda Toolkit, and copy the files over from CuDNN to the toolkit. Check out the videos above for more help on this, however. Also make sure you grab the right versions of Cuda Toolkit and CuDNN. See the installation docs on Tensorflow.org for your operating system to get the version #s you need!

You can also use GPUs in the cloud. I recommend Paperspace for this, and I covered using them in this video

Anywho, let's build the model next.
dense_layers = [0, 1, 2]
layer_sizes = [32, 64, 128]
conv_layers = [1, 2, 3]

    for dense_layer in dense_layers:
        for layer_size in layer_sizes:
            for conv_layer in conv_layers:
                NAME = "{}-conv-{}-nodes-{}-dense-{}".format(conv_layer, layer_size, dense_layer, int(time.time()))
                print(NAME)

                model = Sequential()

                model.add(Conv2D(layer_size, (3, 3), input_shape=X.shape[1:]))
                model.add(Activation('relu'))
                model.add(MaxPooling2D(pool_size=(2, 2)))

                for l in range(conv_layer-1):
                    model.add(Conv2D(layer_size, (3, 3)))
                    model.add(Activation('relu'))
                    model.add(MaxPooling2D(pool_size=(2, 2)))

                model.add(Flatten())
                for _ in range(dense_layer):
                    model.add(Dense(layer_size))
                    model.add(Activation('relu'))

                model.add(Dense(1))
                model.add(Activation('sigmoid'))
                
Even just toying with these parameters will take some significant time. We haven't even begun to touch other concepts like varying layer sizes, activation functions, learning rates, dropouts, and much much more.

In general, I try to test only a few things. You almost want to make a hill-climb operation out of this process. First find a model that works. On this dataset, that part was easy. Then try to tinker with 2 or 3 things max. If there's a long/infinite list of options, such as is the case with layer count and nodes per layer, try to just do maybe one move in each direction.

For example, if you find a 64 node-per-layer is working. Try a 32 and a 128, along with 64, so your list is [32, 64, 128]. If 128 shows better results, then maybe try [64, 128, 256] next, and so on.

Just note that, as you change certain parameters, you may need to revisit older ones. As you tweak dropout, for example, let's say you add or just increase dropout. As you add or increase dropout, you can likely have a larger overall model in terms of layers or nodes per layer than before. A larger model overall might want a larger starting learning rate or a slower rate of decay for the learning rate.

Finally, take note that there is some randomness in models. No two rounds of optimizations will be identical. They should be close, but not identical. Models are also initialized with random weights. This can impact models fairly significantly, especially in shorter numbers of epochs or if you have a small training set.

Anyway, here's the full script for initial model testing:

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
# more info on callbakcs: https://keras.io/callbacks/ model saver is cool too.
from tensorflow.keras.callbacks import TensorBoard
import pickle
import time

pickle_in = open("X.pickle","rb")
X = pickle.load(pickle_in)

pickle_in = open("y.pickle","rb")
y = pickle.load(pickle_in)

X = X/255.0

dense_layers = [0, 1, 2]
layer_sizes = [32, 64, 128]
conv_layers = [1, 2, 3]

for dense_layer in dense_layers:
    for layer_size in layer_sizes:
        for conv_layer in conv_layers:
            NAME = "{}-conv-{}-nodes-{}-dense-{}".format(conv_layer, layer_size, dense_layer, int(time.time()))
            print(NAME)

            model = Sequential()

            model.add(Conv2D(layer_size, (3, 3), input_shape=X.shape[1:]))
            model.add(Activation('relu'))
            model.add(MaxPooling2D(pool_size=(2, 2)))

            for l in range(conv_layer-1):
                model.add(Conv2D(layer_size, (3, 3)))
                model.add(Activation('relu'))
                model.add(MaxPooling2D(pool_size=(2, 2)))

            model.add(Flatten())

            for _ in range(dense_layer):
                model.add(Dense(layer_size))
                model.add(Activation('relu'))

            model.add(Dense(1))
            model.add(Activation('sigmoid'))

            tensorboard = TensorBoard(log_dir="logs/{}".format(NAME))

            model.compile(loss='binary_crossentropy',
                          optimizer='adam',
                          metrics=['accuracy'],
                          )

            model.fit(X, y,
                      batch_size=32,
                      epochs=10,
                      validation_split=0.3,
                      callbacks=[tensorboard])

1-conv-32-nodes-0-dense-1542609160
Train on 17441 samples, validate on 7475 samples
Epoch 1/10
17441/17441 [==============================] - 3s 167us/step - loss: 0.6367 - acc: 0.6352 - val_loss: 0.5891 - val_acc: 0.6958
Epoch 2/10
17441/17441 [==============================] - 2s 127us/step - loss: 0.5701 - acc: 0.7087 - val_loss: 0.5677 - val_acc: 0.7092
Epoch 3/10
17441/17441 [==============================] - 2s 129us/step - loss: 0.5323 - acc: 0.7356 - val_loss: 0.5550 - val_acc: 0.7179
Epoch 4/10
17441/17441 [==============================] - 2s 132us/step - loss: 0.5122 - acc: 0.7557 - val_loss: 0.5618 - val_acc: 0.7108
Epoch 5/10
17441/17441 [==============================] - 2s 130us/step - loss: 0.4896 - acc: 0.7704 - val_loss: 0.5265 - val_acc: 0.7399
Epoch 6/10
17441/17441 [==============================] - 2s 129us/step - loss: 0.4746 - acc: 0.7785 - val_loss: 0.5203 - val_acc: 0.7434
Epoch 7/10
17441/17441 [==============================] - 2s 129us/step - loss: 0.4605 -

17441/17441 [==============================] - 6s 316us/step - loss: 0.3947 - acc: 0.8206 - val_loss: 0.4817 - val_acc: 0.7706
Epoch 7/10
17441/17441 [==============================] - 6s 327us/step - loss: 0.3730 - acc: 0.8334 - val_loss: 0.4486 - val_acc: 0.7862
Epoch 8/10
17441/17441 [==============================] - 6s 335us/step - loss: 0.3513 - acc: 0.8434 - val_loss: 0.4231 - val_acc: 0.8043
Epoch 9/10
17441/17441 [==============================] - 6s 327us/step - loss: 0.3249 - acc: 0.8558 - val_loss: 0.4930 - val_acc: 0.7664
Epoch 10/10
17441/17441 [==============================] - 6s 330us/step - loss: 0.3047 - acc: 0.8660 - val_loss: 0.4223 - val_acc: 0.8111
1-conv-128-nodes-0-dense-1542609390
Train on 17441 samples, validate on 7475 samples
Epoch 1/10
17441/17441 [==============================] - 7s 401us/step - loss: 0.6181 - acc: 0.6578 - val_loss: 0.5779 - val_acc: 0.7064
Epoch 2/10
17441/17441 [==============================] - 7s 392us/step - loss: 0.5445 - acc: 0.7

17441/17441 [==============================] - 4s 201us/step - loss: 0.6653 - acc: 0.5897 - val_loss: 0.6495 - val_acc: 0.6142
Epoch 2/10
17441/17441 [==============================] - 3s 189us/step - loss: 0.5932 - acc: 0.6826 - val_loss: 0.5864 - val_acc: 0.7050
Epoch 3/10
17441/17441 [==============================] - 3s 185us/step - loss: 0.5316 - acc: 0.7316 - val_loss: 0.5132 - val_acc: 0.7454
Epoch 4/10
17441/17441 [==============================] - 3s 183us/step - loss: 0.4874 - acc: 0.7624 - val_loss: 0.4885 - val_acc: 0.7637
Epoch 5/10
17441/17441 [==============================] - 3s 184us/step - loss: 0.4524 - acc: 0.7828 - val_loss: 0.4690 - val_acc: 0.7767
Epoch 6/10
17441/17441 [==============================] - 3s 193us/step - loss: 0.4202 - acc: 0.8039 - val_loss: 0.4957 - val_acc: 0.7624
Epoch 7/10
17441/17441 [==============================] - 3s 194us/step - loss: 0.3937 - acc: 0.8224 - val_loss: 0.4487 - val_acc: 0.7914
Epoch 8/10
17441/17441 [=====================

17441/17441 [==============================] - 11s 640us/step - loss: 0.3436 - acc: 0.8476 - val_loss: 0.4666 - val_acc: 0.7894
Epoch 8/10
17441/17441 [==============================] - 11s 640us/step - loss: 0.3049 - acc: 0.8659 - val_loss: 0.5030 - val_acc: 0.7825
Epoch 9/10
17441/17441 [==============================] - 11s 642us/step - loss: 0.2463 - acc: 0.8996 - val_loss: 0.5380 - val_acc: 0.7853
Epoch 10/10
17441/17441 [==============================] - 11s 639us/step - loss: 0.1905 - acc: 0.9243 - val_loss: 0.5808 - val_acc: 0.7797
3-conv-128-nodes-1-dense-1542610204
Train on 17441 samples, validate on 7475 samples
Epoch 1/10
17441/17441 [==============================] - 11s 644us/step - loss: 0.6371 - acc: 0.6268 - val_loss: 0.5608 - val_acc: 0.7129
Epoch 2/10
17441/17441 [==============================] - 11s 626us/step - loss: 0.5261 - acc: 0.7420 - val_loss: 0.5070 - val_acc: 0.7513
Epoch 3/10
17441/17441 [==============================] - 11s 630us/step - loss: 0.4503 - a

17441/17441 [==============================] - 6s 353us/step - loss: 0.5127 - acc: 0.7503 - val_loss: 0.4983 - val_acc: 0.7609
Epoch 3/10
17441/17441 [==============================] - 6s 364us/step - loss: 0.4662 - acc: 0.7795 - val_loss: 0.4818 - val_acc: 0.7684
Epoch 4/10
17441/17441 [==============================] - 6s 356us/step - loss: 0.4266 - acc: 0.8025 - val_loss: 0.5954 - val_acc: 0.7302
Epoch 5/10
17441/17441 [==============================] - 6s 337us/step - loss: 0.3773 - acc: 0.8313 - val_loss: 0.4661 - val_acc: 0.7830
Epoch 6/10
17441/17441 [==============================] - 6s 360us/step - loss: 0.3263 - acc: 0.8526 - val_loss: 0.5348 - val_acc: 0.7790
Epoch 7/10
17441/17441 [==============================] - 6s 356us/step - loss: 0.2723 - acc: 0.8784 - val_loss: 0.5197 - val_acc: 0.7885
Epoch 8/10
17441/17441 [==============================] - 6s 346us/step - loss: 0.2134 - acc: 0.9111 - val_loss: 0.6153 - val_acc: 0.7591
Epoch 9/10
17441/17441 [=====================

It could be tempting to take the highest validation accuracy model, but I tend to instead go for the best (lowest) validation loss models. Like I said before, there is some randomness when it comes to models, but you should notice trends.

For one, I notice that the models with 0 dense layers seemed to do better overall. There are some very successful models with dense layers, but I am going to guess one is likely not needed here.

So, zooming into the validation accuracy graph, let's check some of the best ones. Here are the top 10:

    3 conv, 64 nodes per layer, 0 dense
    3 conv, 128 nodes per layer, 0 dense
    3 conv, 32 nodes per layer, 0 dense
    3 conv, 32 nodes per layer, 2 dense
    3 conv, 32 nodes per layer, 1 dense
    2 conv, 32 nodes per layer, 0 dense
    2 conv, 64 nodes per layer, 0 dense
    3 conv, 128 nodes per layer, 1 dense
    2 conv, 128 nodes per layer, 0 dense
    2 conv, 32 nodes per layer, 1 dense
    
From here, I think we can be comfortable with 0 dense, and 3 convolutional layers, since every version of those 2 options proved to be better than anything else. Just the top 3 models:
